<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/Tensorflow_Blocks_and_Layesr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

The code is based on the d2l.ai book http://d2l.ai/

In [2]:
### the code is based on d2l.ai book 

In [3]:
### sequence of models , Dense with relu is the first in the chain
net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,activation=tf.nn.relu),
  tf.keras.layers.Dense(10)
])

X=tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.04931409, -0.08198217,  0.282879  ,  0.2841699 , -0.0088197 ,
        -0.4251966 ,  0.27759126, -0.09480061,  0.23644048,  0.3943337 ],
       [ 0.12069568,  0.02676927,  0.21037538,  0.17428683, -0.09368664,
        -0.2026636 ,  0.0821211 , -0.26064616,  0.13454805,  0.06034918]],
      dtype=float32)>

In [4]:
net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.04931409, -0.08198217,  0.282879  ,  0.2841699 , -0.0088197 ,
        -0.4251966 ,  0.27759126, -0.09480061,  0.23644048,  0.3943337 ],
       [ 0.12069568,  0.02676927,  0.21037538,  0.17428683, -0.09368664,
        -0.2026636 ,  0.0821211 , -0.26064616,  0.13454805,  0.06034918]],
      dtype=float32)>

In [5]:
### Custom Block

In [6]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.hidden=tf.keras.layers.Dense(units=256,activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, X):
    return self.out(self.hidden((X)))

In [7]:
net=MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.29268128,  0.26970774, -0.53997064, -0.2227497 ,  0.00782839,
         0.0568753 ,  0.06693274,  0.10750372,  0.03331758, -0.44387406],
       [ 0.2952827 ,  0.04087556, -0.29475218, -0.09809808,  0.02343569,
         0.13902044, -0.10059637, -0.02503465, -0.12724064, -0.3097494 ]],
      dtype=float32)>

In [8]:
class MySequential(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    for module in self.modules:
      X=module(X)
    return(X)


In [9]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.34945226,  0.09425004, -0.44992548, -0.14354521, -0.05716934,
         0.26188537,  0.5844754 ,  0.2377574 , -0.33320907,  0.27498013],
       [ 0.32028604,  0.02710316, -0.4054122 , -0.337573  , -0.05772184,
         0.05528935,  0.27435178,  0.25756633,  0.08833437,  0.2230866 ]],
      dtype=float32)>

Below the Module is constructed where several models are applied in parallel on the same input data and the results are concatenated. It is intended to use the code in multihead attention in Transformers.

In [10]:
#initialize the list of n models which is constructed from the same model
#
def list_of_models(model, n):
  return [model]*n


In [11]:
list_models = [tf.keras.layers.Dense(10)]*3
arg_list = list_of_models(tf.keras.layers.Dense(10),3)

In [12]:
### use *args to supply a list of models of variable length 
class MyParallel(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    list_res= []
    for module in self.modules:
      list_res.append(module(X))
      print(len(list_res))
    print(list_res)
    concat_final= tf.concat(list_res, -1)
    return(concat_final)

In [13]:
net = MyParallel(tf.keras.layers.Dense(10), tf.keras.layers.Dense(10), tf.keras.layers.Dense(10))
X = tf.random.uniform((2,10))
net(X)

1
2
3
[<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.2513558 , -0.27768958,  0.5101268 ,  0.03177354, -0.46417987,
         0.29007128, -0.2668992 , -0.8697561 ,  1.575279  , -0.13892499],
       [-0.5528356 , -0.67307687,  0.31287616,  0.2512623 , -0.66248894,
         0.9035158 , -0.7156859 , -0.8134246 ,  1.3020751 ,  0.4307332 ]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.36709428,  0.40895742, -0.74394673, -1.2246717 , -0.13279912,
        -0.8551988 ,  0.44884032,  0.04543832, -0.6631161 ,  0.644754  ],
       [ 0.18755849,  0.61731833, -0.8863526 , -0.7644035 , -0.3548014 ,
        -0.39990374,  0.3656888 ,  0.4633091 , -0.6807189 ,  0.04607466]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.9445234 ,  0.3418597 ,  0.40121087, -0.69413507, -0.00871749,
         0.8893925 , -0.24391532, -0.36893445, -0.6856817 , -0.05989665],
       [-0.6726131 ,  0.33770025,  0.13621056, -0.15839985,  0.

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[ 0.2513558 , -0.27768958,  0.5101268 ,  0.03177354, -0.46417987,
         0.29007128, -0.2668992 , -0.8697561 ,  1.575279  , -0.13892499,
         0.36709428,  0.40895742, -0.74394673, -1.2246717 , -0.13279912,
        -0.8551988 ,  0.44884032,  0.04543832, -0.6631161 ,  0.644754  ,
        -0.9445234 ,  0.3418597 ,  0.40121087, -0.69413507, -0.00871749,
         0.8893925 , -0.24391532, -0.36893445, -0.6856817 , -0.05989665],
       [-0.5528356 , -0.67307687,  0.31287616,  0.2512623 , -0.66248894,
         0.9035158 , -0.7156859 , -0.8134246 ,  1.3020751 ,  0.4307332 ,
         0.18755849,  0.61731833, -0.8863526 , -0.7644035 , -0.3548014 ,
        -0.39990374,  0.3656888 ,  0.4633091 , -0.6807189 ,  0.04607466,
        -0.6726131 ,  0.33770025,  0.13621056, -0.15839985,  0.37523422,
         0.670637  ,  0.34844506, -0.09180941, -0.92587626, -0.3934785 ]],
      dtype=float32)>

In [14]:
### to use the list_models (which is a list) we have to unpack it
net = MyParallel(*list_models)
X = tf.random.uniform((2,10))
net(X)

1
2
3
[<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.6369225 ,  0.23359372,  0.1602743 , -0.31942943, -0.40882462,
        -0.02672926, -0.44032443,  0.01279652, -0.63664097, -0.445827  ],
       [ 0.1844315 ,  0.4043808 ,  0.3806227 , -0.5626929 , -0.5221994 ,
         0.25772053, -0.07888096, -0.03687203, -0.5237836 , -0.27514204]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.6369225 ,  0.23359372,  0.1602743 , -0.31942943, -0.40882462,
        -0.02672926, -0.44032443,  0.01279652, -0.63664097, -0.445827  ],
       [ 0.1844315 ,  0.4043808 ,  0.3806227 , -0.5626929 , -0.5221994 ,
         0.25772053, -0.07888096, -0.03687203, -0.5237836 , -0.27514204]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.6369225 ,  0.23359372,  0.1602743 , -0.31942943, -0.40882462,
        -0.02672926, -0.44032443,  0.01279652, -0.63664097, -0.445827  ],
       [ 0.1844315 ,  0.4043808 ,  0.3806227 , -0.5626929 , -0.

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[ 0.6369225 ,  0.23359372,  0.1602743 , -0.31942943, -0.40882462,
        -0.02672926, -0.44032443,  0.01279652, -0.63664097, -0.445827  ,
         0.6369225 ,  0.23359372,  0.1602743 , -0.31942943, -0.40882462,
        -0.02672926, -0.44032443,  0.01279652, -0.63664097, -0.445827  ,
         0.6369225 ,  0.23359372,  0.1602743 , -0.31942943, -0.40882462,
        -0.02672926, -0.44032443,  0.01279652, -0.63664097, -0.445827  ],
       [ 0.1844315 ,  0.4043808 ,  0.3806227 , -0.5626929 , -0.5221994 ,
         0.25772053, -0.07888096, -0.03687203, -0.5237836 , -0.27514204,
         0.1844315 ,  0.4043808 ,  0.3806227 , -0.5626929 , -0.5221994 ,
         0.25772053, -0.07888096, -0.03687203, -0.5237836 , -0.27514204,
         0.1844315 ,  0.4043808 ,  0.3806227 , -0.5626929 , -0.5221994 ,
         0.25772053, -0.07888096, -0.03687203, -0.5237836 , -0.27514204]],
      dtype=float32)>

In [15]:
net = MyParallel(*arg_list)
X = tf.random.uniform((2,10))
net(X)

1
2
3
[<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.53765035,  0.6969004 , -0.0586617 , -0.5094317 , -0.06634474,
         0.21278669,  0.1946829 ,  0.34302768,  0.60614115,  0.6390437 ],
       [ 0.65262216,  0.8320782 , -0.01154299, -0.28898478, -0.12834722,
         0.08020669,  0.09392177,  0.6357094 ,  0.3128935 , -0.17434597]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.53765035,  0.6969004 , -0.0586617 , -0.5094317 , -0.06634474,
         0.21278669,  0.1946829 ,  0.34302768,  0.60614115,  0.6390437 ],
       [ 0.65262216,  0.8320782 , -0.01154299, -0.28898478, -0.12834722,
         0.08020669,  0.09392177,  0.6357094 ,  0.3128935 , -0.17434597]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.53765035,  0.6969004 , -0.0586617 , -0.5094317 , -0.06634474,
         0.21278669,  0.1946829 ,  0.34302768,  0.60614115,  0.6390437 ],
       [ 0.65262216,  0.8320782 , -0.01154299, -0.28898478, -0.

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.53765035,  0.6969004 , -0.0586617 , -0.5094317 , -0.06634474,
         0.21278669,  0.1946829 ,  0.34302768,  0.60614115,  0.6390437 ,
        -0.53765035,  0.6969004 , -0.0586617 , -0.5094317 , -0.06634474,
         0.21278669,  0.1946829 ,  0.34302768,  0.60614115,  0.6390437 ,
        -0.53765035,  0.6969004 , -0.0586617 , -0.5094317 , -0.06634474,
         0.21278669,  0.1946829 ,  0.34302768,  0.60614115,  0.6390437 ],
       [ 0.65262216,  0.8320782 , -0.01154299, -0.28898478, -0.12834722,
         0.08020669,  0.09392177,  0.6357094 ,  0.3128935 , -0.17434597,
         0.65262216,  0.8320782 , -0.01154299, -0.28898478, -0.12834722,
         0.08020669,  0.09392177,  0.6357094 ,  0.3128935 , -0.17434597,
         0.65262216,  0.8320782 , -0.01154299, -0.28898478, -0.12834722,
         0.08020669,  0.09392177,  0.6357094 ,  0.3128935 , -0.17434597]],
      dtype=float32)>